# Week 4: Advanced Continuous Distributions

**Course:** Statistics for Data Science II (BSMA1004)  
**Week:** 4 of 12

## Learning Objectives
- Master exponential and gamma distributions
- Understand beta distribution applications
- Work with joint continuous distributions
- Apply transformations of random variables

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from scipy.special import gamma as gamma_func

np.random.seed(42)
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

print('✓ Libraries loaded')

## 1. Exponential Distribution

### Definition
Models time between events in a Poisson process

**PDF:** $f(x) = \lambda e^{-\lambda x}$ for $x \geq 0$

**Properties:**
- $E[X] = \frac{1}{\lambda}$ (mean time between events)
- $\text{Var}(X) = \frac{1}{\lambda^2}$
- **Memoryless property:** $P(X > s+t | X > s) = P(X > t)$

In [ ]:
# Exponential distribution examples
lambdas = [0.5, 1.0, 2.0]
x = np.linspace(0, 8, 1000)

fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# PDF comparison
for lam in lambdas:
    pdf = stats.expon.pdf(x, scale=1/lam)
    axes[0].plot(x, pdf, label=f'λ={lam}, μ={1/lam:.1f}', linewidth=2)
axes[0].set_xlabel('x', fontsize=12)
axes[0].set_ylabel('f(x)', fontsize=12)
axes[0].set_title('Exponential PDF', fontweight='bold')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# CDF comparison
for lam in lambdas:
    cdf = stats.expon.cdf(x, scale=1/lam)
    axes[1].plot(x, cdf, label=f'λ={lam}', linewidth=2)
axes[1].set_xlabel('x', fontsize=12)
axes[1].set_ylabel('F(x)', fontsize=12)
axes[1].set_title('Exponential CDF', fontweight='bold')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Application: Customer service time
lam = 0.2  # customers per minute
print(f"\n📊 Customer Service Example (λ={lam}/min)")
print(f"Average time between customers: {1/lam:.1f} minutes")
print(f"P(wait < 3 min) = {stats.expon.cdf(3, scale=1/lam):.3f}")
print(f"P(wait > 10 min) = {1 - stats.expon.cdf(10, scale=1/lam):.3f}")

## 2. Gamma Distribution

### Definition
Generalizes exponential: sum of k independent Exponential(λ) random variables

**PDF:** $f(x) = \frac{\lambda^k x^{k-1} e^{-\lambda x}}{\Gamma(k)}$ for $x \geq 0$

**Parameters:**
- Shape parameter k (k=1 gives Exponential)
- Rate parameter λ

**Properties:**
- $E[X] = \frac{k}{\lambda}$
- $\text{Var}(X) = \frac{k}{\lambda^2}$

In [ ]:
# Gamma distribution examples
lam = 1
shapes = [1, 2, 5, 9]
x = np.linspace(0, 20, 1000)

plt.figure(figsize=(12, 6))
for k in shapes:
    pdf = stats.gamma.pdf(x, a=k, scale=1/lam)
    plt.plot(x, pdf, label=f'k={k}, E[X]={k/lam}', linewidth=2)

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Gamma Distribution PDF', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Verify: Sum of k exponentials
k = 5
lam = 0.5
n_sim = 10000
exp_rvs = [np.random.exponential(1/lam, n_sim) for _ in range(k)]
sum_exp = np.sum(exp_rvs, axis=0)
gamma_rv = np.random.gamma(k, 1/lam, n_sim)

print(f"\n📊 Verification: Sum of {k} Exponential(λ={lam})")
print(f"E[Sum] = {sum_exp.mean():.3f} (theory: {k/lam:.3f})")
print(f"Gamma({k}, {lam}): E[X] = {gamma_rv.mean():.3f}")

## 3. Beta Distribution

### Definition
Models probabilities and proportions on (0,1)

**PDF:** $f(x) = \frac{x^{\alpha-1}(1-x)^{\beta-1}}{B(\alpha,\beta)}$ for $0 \leq x \leq 1$

**Properties:**
- $E[X] = \frac{\alpha}{\alpha+\beta}$
- $\text{Mode} = \frac{\alpha-1}{\alpha+\beta-2}$ (for α,β > 1)

**Applications:** Bayesian priors, click-through rates, success probabilities

In [ ]:
# Beta distribution shapes
params = [(0.5, 0.5), (2, 2), (5, 2), (2, 5), (8, 4)]
x = np.linspace(0, 1, 1000)

plt.figure(figsize=(14, 7))
for alpha, beta in params:
    pdf = stats.beta.pdf(x, alpha, beta)
    mean = alpha / (alpha + beta)
    plt.plot(x, pdf, label=f'α={alpha}, β={beta}, E[X]={mean:.2f}', linewidth=2)

plt.xlabel('x', fontsize=12)
plt.ylabel('f(x)', fontsize=12)
plt.title('Beta Distribution PDF (Various Shapes)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

# Application: Website conversion rate
alpha, beta = 30, 270  # 30 successes, 270 failures
mean = alpha / (alpha + beta)
mode = (alpha - 1) / (alpha + beta - 2)
ci_lower = stats.beta.ppf(0.025, alpha, beta)
ci_upper = stats.beta.ppf(0.975, alpha, beta)

print(f"\n📊 Website Conversion Rate (α={alpha}, β={beta})")
print(f"Mean estimate: {mean:.3f} ({mean*100:.1f}%)")
print(f"Mode: {mode:.3f}")
print(f"95% CI: [{ci_lower:.3f}, {ci_upper:.3f}]")

## 4. Joint Continuous Distributions

### Definition
Joint PDF f(x,y) satisfies:
1. $f(x,y) \geq 0$
2. $\int_{-\infty}^{\infty} \int_{-\infty}^{\infty} f(x,y) dx dy = 1$
3. $P((X,Y) \in A) = \iint_A f(x,y) dx dy$

**Marginals:** $f_X(x) = \int_{-\infty}^{\infty} f(x,y) dy$

In [ ]:
# Bivariate normal distribution
mean = [0, 0]
rho = 0.7
cov = [[1, rho], [rho, 1]]

# Create grid
x = np.linspace(-3, 3, 100)
y = np.linspace(-3, 3, 100)
X, Y = np.meshgrid(x, y)
pos = np.dstack((X, Y))

# Compute PDF
rv = stats.multivariate_normal(mean, cov)
Z = rv.pdf(pos)

# Visualize
fig = plt.figure(figsize=(16, 5))

# 3D surface
ax1 = fig.add_subplot(131, projection='3d')
ax1.plot_surface(X, Y, Z, cmap='viridis', alpha=0.8)
ax1.set_xlabel('X')
ax1.set_ylabel('Y')
ax1.set_zlabel('f(x,y)')
ax1.set_title('Joint PDF', fontweight='bold')

# Contour plot
ax2 = fig.add_subplot(132)
contour = ax2.contourf(X, Y, Z, levels=20, cmap='viridis')
plt.colorbar(contour, ax=ax2)
ax2.set_xlabel('X', fontsize=12)
ax2.set_ylabel('Y', fontsize=12)
ax2.set_title('Contour Plot', fontweight='bold')
ax2.grid(True, alpha=0.3)

# Samples
ax3 = fig.add_subplot(133)
samples = rv.rvs(1000)
ax3.scatter(samples[:, 0], samples[:, 1], alpha=0.5, s=20)
ax3.set_xlabel('X', fontsize=12)
ax3.set_ylabel('Y', fontsize=12)
ax3.set_title(f'Samples (ρ={rho})', fontweight='bold')
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Transformations of Random Variables

### Method 1: CDF Method
If Y = g(X), then $F_Y(y) = P(Y \leq y) = P(g(X) \leq y)$

### Method 2: Jacobian Method (for monotone transformations)
$$f_Y(y) = f_X(x) \left| \frac{dx}{dy} \right|$$

In [ ]:
# Example 1: Y = X² where X ~ N(0,1)
n_samples = 10000
X = np.random.normal(0, 1, n_samples)
Y = X**2

# Y follows chi-square distribution with 1 df
x_range = np.linspace(0, 10, 1000)
chi2_pdf = stats.chi2.pdf(x_range, df=1)

plt.figure(figsize=(12, 6))
plt.hist(Y, bins=100, density=True, alpha=0.7, edgecolor='black', label='Y=X² samples')
plt.plot(x_range, chi2_pdf, 'r-', linewidth=2, label='χ²(1) PDF')
plt.xlabel('y', fontsize=12)
plt.ylabel('Density', fontsize=12)
plt.title('Transformation: Y = X², X ~ N(0,1)', fontsize=14, fontweight='bold')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 10])
plt.tight_layout()
plt.show()

print(f"E[Y] simulation: {Y.mean():.3f}")
print(f"E[Y] theory (χ² df=1): {1:.3f} ✓")

## 6. Practice Problems

### Problem 1
Exponential(λ=0.5): Find P(X > 2) and median

### Problem 2
Beta(α=5, β=2): Find E[X] and Mode

### Problem 3
If X ~ Uniform(0,1), find PDF of Y = -ln(X)

In [ ]:
print("="*60)
print("SOLUTIONS")
print("="*60)

# Problem 1
lam = 0.5
print(f"\n📝 Problem 1: Exponential(λ={lam})")
prob = 1 - stats.expon.cdf(2, scale=1/lam)
median = stats.expon.median(scale=1/lam)
print(f"P(X > 2) = e^(-λ·2) = e^(-{lam}·2) = {prob:.4f}")
print(f"Median = ln(2)/λ = ln(2)/{lam} = {median:.4f}")

# Problem 2
alpha, beta = 5, 2
print(f"\n📝 Problem 2: Beta(α={alpha}, β={beta})")
mean = alpha / (alpha + beta)
mode = (alpha - 1) / (alpha + beta - 2)
print(f"E[X] = α/(α+β) = {alpha}/({alpha}+{beta}) = {mean:.4f}")
print(f"Mode = (α-1)/(α+β-2) = ({alpha}-1)/({alpha}+{beta}-2) = {mode:.4f}")

# Problem 3
print(f"\n📝 Problem 3: Y = -ln(X), X ~ Uniform(0,1)")
print("Y follows Exponential(λ=1)")
print("Proof: F_Y(y) = P(-ln(X) ≤ y) = P(X ≥ e^(-y)) = 1 - e^(-y)")
print("Therefore f_Y(y) = e^(-y) for y ≥ 0 (Exponential(1))")

print("\n" + "="*60)

## 7. Summary & Key Takeaways

### 📚 Core Concepts
1. **Exponential**: Memoryless, time between events
2. **Gamma**: Sum of k exponentials, flexible shape
3. **Beta**: Probabilities on (0,1), Bayesian priors
4. **Joint distributions**: f(x,y), marginals, conditionals
5. **Transformations**: CDF and Jacobian methods

### 🔑 Key Formulas
- Exponential: $E[X] = 1/\lambda$, memoryless property
- Gamma: $E[X] = k/\lambda$
- Beta: $E[X] = \alpha/(\alpha+\beta)$

### 🚀 Next Week
**Week 5: Sampling Methods**

---
**🎓 End of Week 4**